In [1]:
import math
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import time
import datetime
import random
random.seed(1234)

from scipy import interp
import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from functools import reduce
from tqdm import tqdm, trange
from copy import deepcopy

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
seed = 19961231
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
hla_sequence = pd.read_csv('/home/chujunyi/5_ZY_MHC/data/common_hla_sequence.csv')

# 定义模型

In [21]:
def make_data(data):
    pep_inputs, hla_inputs = [], []
    if 'peptide' not in data.columns: 
        peptides = data.mutation_peptide
    else:
        peptides = data.peptide
        
    for pep, hla in zip(peptides, data.HLA_sequence):
        pep, hla = pep.ljust(pep_max_len, '-'), hla.ljust(hla_max_len, '-')
        pep_input = [[vocab[n] for n in pep]] # [[1, 2, 3, 4, 0], [1, 2, 3, 5, 0]]
        hla_input = [[vocab[n] for n in hla]]
        pep_inputs.extend(pep_input)
        hla_inputs.extend(hla_input)
    return torch.LongTensor(pep_inputs), torch.LongTensor(hla_inputs)

class MyDataSet(Data.Dataset):
    def __init__(self, pep_inputs, hla_inputs):
        super(MyDataSet, self).__init__()
        self.pep_inputs = pep_inputs
        self.hla_inputs = hla_inputs

    def __len__(self): # 样本数
        return self.pep_inputs.shape[0] # 改成hla_inputs也可以哦！

    def __getitem__(self, idx):
        return self.pep_inputs[idx], self.hla_inputs[idx]

In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        '''
        x: [seq_len, batch_size, d_model]
        '''
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    seq_len could be src_len or it could be tgt_len
    seq_len in seq_q and seq_len in seq_k maybe not equal
    '''
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size, 1, len_k], False is masked
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size, len_q, len_k]

class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        '''
        Q: [batch_size, n_heads, len_q, d_k]
        K: [batch_size, n_heads, len_k, d_k]
        V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, seq_len, seq_len]
        '''
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size, n_heads, len_q, len_k]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is True.
        
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V) # [batch_size, n_heads, len_q, d_v]
        return context, attn


In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device("cuda" if self.use_cuda else "cpu")
        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)
    def forward(self, input_Q, input_K, input_V, attn_mask):
        '''
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
        residual, batch_size = input_Q, input_Q.size(0)
        # (B, S, D) -proj-> (B, S, D_new) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # Q: [batch_size, n_heads, len_q, d_k]
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # V: [batch_size, n_heads, len_v(=len_k), d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]

        # context: [batch_size, n_heads, len_q, d_v], attn: [batch_size, n_heads, len_q, len_k]
        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)
        context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v) # context: [batch_size, len_q, n_heads * d_v]
        output = self.fc(context) # [batch_size, len_q, d_model]
        return nn.LayerNorm(d_model).to(device)(output + residual), attn

In [7]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device("cuda" if self.use_cuda else "cpu")
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False)
        )
    def forward(self, inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model).to(device)(output + residual) # [batch_size, seq_len, d_model]


In [8]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        '''
        enc_inputs: [batch_size, src_len, d_model]
        enc_self_attn_mask: [batch_size, src_len, src_len]
        '''
        # enc_outputs: [batch_size, src_len, d_model], attn: [batch_size, n_heads, src_len, src_len]
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs, attn


# In[20]:


class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        '''
        enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs) # [batch_size, src_len, src_len]
        enc_self_attns = []
        for layer in self.layers:
            # enc_outputs: [batch_size, src_len, d_model], enc_self_attn: [batch_size, n_heads, src_len, src_len]
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns


# ### Decoder

# In[21]:


class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, dec_self_attn_mask): # dec_inputs = enc_outputs
        '''
        dec_inputs: [batch_size, tgt_len, d_model]
        enc_outputs: [batch_size, src_len, d_model]
        dec_self_attn_mask: [batch_size, tgt_len, tgt_len]
        '''
        # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len]
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)
        dec_outputs = self.pos_ffn(dec_outputs) # [batch_size, tgt_len, d_model]
        return dec_outputs, dec_self_attn


# In[22]:


class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
#         self.tgt_emb = nn.Embedding(d_model * 2, d_model)
        self.use_cuda = use_cuda
        device = torch.device("cuda" if self.use_cuda else "cpu")
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])
        self.tgt_len = tgt_len
        
    def forward(self, dec_inputs): # dec_inputs = enc_outputs (batch_size, peptide_hla_maxlen_sum, d_model)
        '''
        dec_inputs: [batch_size, tgt_len]
        enc_intpus: [batch_size, src_len]
        enc_outputs: [batsh_size, src_len, d_model]
        '''
#         dec_outputs = self.tgt_emb(dec_inputs) # [batch_size, tgt_len, d_model]
        dec_outputs = self.pos_emb(dec_inputs.transpose(0, 1)).transpose(0, 1).to(device) # [batch_size, tgt_len, d_model]
#         dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs).cuda() # [batch_size, tgt_len, tgt_len]
        dec_self_attn_pad_mask = torch.LongTensor(np.zeros((dec_inputs.shape[0], tgt_len, tgt_len))).bool().to(device)

        dec_self_attns = []
        for layer in self.layers:
            # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
            dec_outputs, dec_self_attn = layer(dec_outputs, dec_self_attn_pad_mask)
            dec_self_attns.append(dec_self_attn)
            
        return dec_outputs, dec_self_attns

In [9]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device("cuda" if use_cuda else "cpu")
        self.pep_encoder = Encoder().to(device)
        self.hla_encoder = Encoder().to(device)
        self.decoder = Decoder().to(device)
        self.tgt_len = tgt_len
        self.projection = nn.Sequential(
                                        nn.Linear(tgt_len * d_model, 256),
                                        nn.ReLU(True),

                                        nn.BatchNorm1d(256),
                                        nn.Linear(256, 64),
                                        nn.ReLU(True),

                                        #output layer
                                        nn.Linear(64, 2)
                                        ).to(device)
        
    def forward(self, pep_inputs, hla_inputs):
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        '''
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)
        
        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        pep_enc_outputs, pep_enc_self_attns = self.pep_encoder(pep_inputs)
        hla_enc_outputs, hla_enc_self_attns = self.hla_encoder(hla_inputs)
        enc_outputs = torch.cat((pep_enc_outputs, hla_enc_outputs), 1) # concat pep & hla embedding
        
        # dec_outpus: [batch_size, tgt_len, d_model], dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        dec_outputs, dec_self_attns = self.decoder(enc_outputs)
        dec_outputs = dec_outputs.view(dec_outputs.shape[0], -1) # Flatten [batch_size, tgt_len * d_model]
        dec_logits = self.projection(dec_outputs) # dec_logits: [batch_size, tgt_len, tgt_vocab_size]

        return dec_logits.view(-1, dec_logits.size(-1)), pep_enc_self_attns, hla_enc_self_attns, dec_self_attns

In [55]:
def performances(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('acc={:.4f}'.format(accuracy))
    
    return (fn, tp, accuracy)


# In[25]:


def transfer(y_prob, threshold = 0.5):
    return np.array([[0, 1][x > threshold] for x in y_prob])


# In[26]:


f_mean = lambda l: sum(l)/len(l)


# In[28]:


def performances_to_pd(performances_list):
    metrics_name = ['roc_auc', 'accuracy', 'mcc', 'f1', 'sensitivity', 'specificity', 'precision', 'recall', 'aupr']

    performances_pd = pd.DataFrame(performances_list, columns = metrics_name)
    performances_pd.loc['mean'] = performances_pd.mean(axis = 0)
    performances_pd.loc['std'] = performances_pd.std(axis = 0)
    
    return performances_pd


In [63]:
def eval_step(model, val_loader, threshold = 0.5, use_cuda = False):
    device = torch.device("cuda" if use_cuda else "cpu")
    
    model.eval()
    torch.manual_seed(19961231)
    torch.cuda.manual_seed(19961231)
    with torch.no_grad():
        y_prob_val_list, dec_attns_val_list = [], []
        for val_pep_inputs, val_hla_inputs in tqdm(val_loader):
            val_pep_inputs, val_hla_inputs = val_pep_inputs.to(device), val_hla_inputs.to(device)
            val_outputs, _, _, val_dec_self_attns = model(val_pep_inputs, val_hla_inputs)

            y_prob_val = nn.Softmax(dim = 1)(val_outputs)[:, 1].cpu().detach().numpy()
            y_prob_val_list.extend(y_prob_val)
            
            dec_attns_val_list.extend(val_dec_self_attns[0][:, :, 15:, :15]) # 只要（34,15）行HLA，列peptide
                    
        y_pred_val_list = transfer(y_prob_val_list, threshold)
        metrics_val = performances([1] * len(y_pred_val_list), y_pred_val_list, y_prob_val_list, print_ = True)
    return y_pred_val_list, y_prob_val_list, metrics_val

In [12]:
pep_max_len = 15 # peptide; enc_input max sequence length
hla_max_len = 34 # hla; dec_input(=dec_output) max sequence length
tgt_len = pep_max_len + hla_max_len
pep_max_len, hla_max_len

vocab = np.load('Transformer_vocab_dict.npy', allow_pickle = True).item()
vocab_size = len(vocab)


# In[36]:


# Transformer Parameters
d_model = 64  # Embedding Size
d_ff = 512 # FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_layers = 2  # number of Encoder of Decoder Layer

batch_size = 1024
epochs = 50
threshold = 0.5

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# 获取HLA sequence

In [22]:
neoantigen = pd.read_csv('./data/合并整理-新抗原.csv', sep = ',').drop_duplicates(keep = 'first')
hpv_vaccine = pd.read_csv('./data/HPV病毒疫苗表位.csv', sep = ',').drop_duplicates(keep = 'first')
print(neoantigen.shape, hpv_vaccine.shape)

neoantigen['Length'] = [len(item) for item in neoantigen.peptide]
hpv_vaccine['Length'] = [len(item) for item in hpv_vaccine.peptide]

neoantigen['length'] = neoantigen['Length']
hpv_vaccine['length'] = hpv_vaccine['Length']

neoantigen['label'] = 1
hpv_vaccine['label'] = 1

neoantigen.peptide = neoantigen.peptide.str.upper()
hpv_vaccine.peptide = hpv_vaccine.peptide.str.upper()

neoantigen = pd.merge(neoantigen, hla_sequence, on = 'HLA').drop_duplicates(keep = 'first')
hpv_vaccine = pd.merge(hpv_vaccine, hla_sequence, on = 'HLA').drop_duplicates(keep = 'first')

print(neoantigen.shape, hpv_vaccine.shape)
neoantigen.head(), hpv_vaccine.head()

(232, 4) (278, 2)
(221, 7) (278, 6)


(      peptide          HLA  Length  Mut_pos  length  label  \
 0  PSDTRQMLFY  HLA-A*01:01      10        2      10      1   
 1  LTDDRLFTCY  HLA-A*01:01      10       10      10      1   
 2   YTDFHCQYV  HLA-A*01:01       9        5       9      1   
 3  YTDFHCQYVK  HLA-A*01:01      10        5      10      1   
 4   FSADIFFFF  HLA-A*01:01       9        6       9      1   
 
                          HLA_sequence  
 0  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  
 1  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  
 2  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  
 3  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  
 4  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  ,
       peptide          HLA  Length  length  label  \
 0   ISEYRHYCY  HLA-A*01:01       9       9      1   
 1  HGDTPTLHEY  HLA-A*01:01      10      10      1   
 2  DLQPETTDLY  HLA-A*01:01      10      10      1   
 3  AVCDKCLKFY  HLA-A*01:01      10      10      1   
 4  LKFYSKISEY  HLA-A*01:01      10      10      1   
 
                          HLA_sequence  
 0  YF

In [23]:
Counter(neoantigen.Length), '\n', Counter(hpv_vaccine.Length)

(Counter({10: 70, 9: 130, 11: 13, 8: 7, 12: 1}),
 '\n',
 Counter({9: 79, 10: 80, 8: 43, 11: 76}))

# 获得数据

In [25]:
neoantigen_pep_inputs, neoantigen_hla_inputs = make_data(neoantigen)
hpv_pep_inputs, hpv_hla_inputs = make_data(hpv_vaccine)

neoantigen_loader = Data.DataLoader(MyDataSet(neoantigen_pep_inputs, neoantigen_hla_inputs), batch_size = len(neoantigen), shuffle = False, num_workers = 0)
hpv_loader = Data.DataLoader(MyDataSet(hpv_pep_inputs, hpv_hla_inputs), batch_size = len(hpv_vaccine), shuffle = False, num_workers = 0)

# 运行

In [67]:
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
n_layers, n_heads = 1, 9
model_pwd = '/home/chujunyi/5_ZY_MHC/model/pHLAIformer/'
model_files = ['model_layer1_multihead9_fold0.pkl',
              'model_layer1_multihead9_fold1.pkl',
              'model_layer1_multihead9_fold2.pkl',
              'model_layer1_multihead9_fold3.pkl',
              'model_layer1_multihead9_fold4.pkl']

neoantigen_metrics_dict, hpv_metrics_dict = {}, {}

for idx, model_file in enumerate(model_files):
    fold = idx
    temp =  str(n_layers) + '-' + str(n_heads) + '-' + str(fold)
    model_eval = Transformer().to(device)
    model_eval.load_state_dict(torch.load(model_pwd + model_file), strict = True)

    model_eval.eval()
    print('\n****neoantigen****')
    y_pred, y_prob, metrics_res_neoantigen = eval_step(model_eval, neoantigen_loader, 0.5, use_cuda)
    neoantigen['y_pred'], neoantigen['y_prob'] = y_pred, y_prob
    neoantigen = neoantigen.round({'y_prob': 4})
    print(Counter(neoantigen.y_pred))
    neoantigen_metrics_dict[temp] = metrics_res_neoantigen

    print('\n****hpv vaccinate****')
    y_pred, y_prob, metrics_res_hpv = eval_step(model_eval, hpv_loader, 0.5, use_cuda)
    hpv_vaccine['y_pred'], hpv_vaccine['y_prob'] = y_pred, y_prob
    hpv_vaccine = hpv_vaccine.round({'y_prob': 4})
    print(Counter(hpv_vaccine.y_pred))
    hpv_metrics_dict[temp] = metrics_res_hpv


  0%|          | 0/1 [00:00<?, ?it/s]


****neoantigen****
tn = 0, fp = 0, fn = 9, tp = 212
y_pred: 0 = 9 | 1 = 212
y_true: 0 = 0 | 1 = 221
acc=0.9593
Counter({1: 212, 0: 9})

****hpv vaccinate****
tn = 0, fp = 0, fn = 106, tp = 172
y_pred: 0 = 106 | 1 = 172
y_true: 0 = 0 | 1 = 278
acc=0.6187
Counter({1: 172, 0: 106})

****neoantigen****


  0%|          | 0/1 [00:00<?, ?it/s]

tn = 0, fp = 0, fn = 9, tp = 212
y_pred: 0 = 9 | 1 = 212
y_true: 0 = 0 | 1 = 221
acc=0.9593
Counter({1: 212, 0: 9})

****hpv vaccinate****
tn = 0, fp = 0, fn = 97, tp = 181
y_pred: 0 = 97 | 1 = 181
y_true: 0 = 0 | 1 = 278
acc=0.6511
Counter({1: 181, 0: 97})

****neoantigen****
tn = 0, fp = 0, fn = 8, tp = 213
y_pred: 0 = 8 | 1 = 213
y_true: 0 = 0 | 1 = 221
acc=0.9638
Counter({1: 213, 0: 8})

****hpv vaccinate****


100%|██████████| 1/1 [00:00<00:00, 17.70it/s]


tn = 0, fp = 0, fn = 89, tp = 189
y_pred: 0 = 89 | 1 = 189
y_true: 0 = 0 | 1 = 278
acc=0.6799
Counter({1: 189, 0: 89})

****neoantigen****
tn = 0, fp = 0, fn = 9, tp = 212
y_pred: 0 = 9 | 1 = 212
y_true: 0 = 0 | 1 = 221
acc=0.9593
Counter({1: 212, 0: 9})

****hpv vaccinate****
tn = 0, fp = 0, fn = 104, tp = 174
y_pred: 0 = 104 | 1 = 174
y_true: 0 = 0 | 1 = 278
acc=0.6259
Counter({1: 174, 0: 104})


100%|██████████| 1/1 [00:00<00:00, 19.77it/s]


****neoantigen****
tn = 0, fp = 0, fn = 12, tp = 209
y_pred: 0 = 12 | 1 = 209
y_true: 0 = 0 | 1 = 221
acc=0.9457
Counter({1: 209, 0: 12})

****hpv vaccinate****
tn = 0, fp = 0, fn = 101, tp = 177
y_pred: 0 = 101 | 1 = 177
y_true: 0 = 0 | 1 = 278
acc=0.6367
Counter({1: 177, 0: 101})


In [68]:
neoantigen_metrics_dict, hpv_metrics_dict

({'1-9-0': (9, 212, 0.9592760180995475),
  '1-9-1': (9, 212, 0.9592760180995475),
  '1-9-2': (8, 213, 0.9638009049773756),
  '1-9-3': (9, 212, 0.9592760180995475),
  '1-9-4': (12, 209, 0.9457013574660633)},
 {'1-9-0': (106, 172, 0.6187050359712231),
  '1-9-1': (97, 181, 0.6510791366906474),
  '1-9-2': (89, 189, 0.6798561151079137),
  '1-9-3': (104, 174, 0.6258992805755396),
  '1-9-4': (101, 177, 0.6366906474820144)})

# 其他方法的sh处理

In [34]:
methods = ['ann', 
           'comblib_sidney2008', 
           'consensus', 
           'netmhccons', 
           'netmhcpan_ba', 
           'netmhcpan_el', 
           'netmhcstabpan', 
           'pickpocket', 
           'smm', 
           'smmpmbec',
           'anthem',
           'transformer']

In [35]:
hla_methods_dict = {}
for method in methods[:-2]:
    hlas_filename = '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/api_provide_hlas/' + method + '.hlas'
    with open(hlas_filename, 'r') as f:
        hlas = f.readlines()
        hlas = [item.replace('\n', '') for item in hlas[1:-3]]
    hla_methods_dict[method] = hlas
    print('Method = {} | # HLAs = {}'.format(method, len(hlas)))

Method = ann | # HLAs = 89
Method = comblib_sidney2008 | # HLAs = 14
Method = consensus | # HLAs = 81
Method = netmhccons | # HLAs = 2915
Method = netmhcpan_ba | # HLAs = 10386
Method = netmhcpan_el | # HLAs = 10386
Method = netmhcstabpan | # HLAs = 2915
Method = pickpocket | # HLAs = 2924
Method = smm | # HLAs = 81
Method = smmpmbec | # HLAs = 83


### 除了anthem和transformer以外的

In [36]:
# def curl_strs(method, eval_data, length, hla, output_name, file_num):
#     s1 = 'curl --data "method=' + method
#     s2 = '&sequence_text='
#     s3 = '&allele=' + hla
#     s4 = '&length=' + str(length)
#     s5 = '" http://tools-cluster-interface.iedb.org/tools_api/mhci/'
    
#     output_pwd = '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/output_file/' + method + '/'
#     s6 = ' > ' + output_pwd + output_name + '_length' + str(length) + '_' + hla + '_' + str(file_num) + '.output'
#     if not os.path.exists(output_pwd):
#         os.makedirs(output_pwd)
        
#     for idx in range(eval_data.shape[0]):
#         item = eval_data.iloc[idx,:]
#         peptide, hla, length = item.peptide, item.HLA, item.Length
#         if idx != len(eval_data)-1:
#             s2 += '%3E' + 'peptide' + str(idx+1) + '%0A' + peptide + '%0A'
#         else:
#             s2 += '%3E' + 'peptide' + str(idx+1) + '%0A' + peptide
#     s = s1 + s2 + s3 + s4 + s5 + s6
#     return s

In [37]:
# def curl_sh_file(eval_data, hla_methods_dict, eval_type = 'experimental', method = 'ann', threshold = 3000):
#     sh_str = ''
#     for length in range(8, 15):
#         data_length = eval_data[eval_data.Length == length]
#         if data_length.shape[0] == 0: continue
            
#         hlas = list(set(data_length.HLA).intersection(set(hla_methods_dict[method])))
#         for hla in tqdm(hlas):
#             data_length_hla = data_length[data_length.HLA == hla]
#             file_num = int(len(data_length_hla) / threshold) + 1
#             if file_num == 1:
#                 s = curl_strs(method, data_length_hla, length, hla, eval_type, file_num = 0)
#                 sh_str += s + '\n'
#             else:
#                 for i in range(file_num):
#                     start, end = i * threshold, (i+1) * threshold
#                     if i != file_num - 1:
#                         s = curl_strs(method, data_length_hla.iloc[start:end, :], length, hla, eval_type, file_num = i)
#                     else:
#                         s = curl_strs(method, data_length_hla.iloc[start:, :], length, hla, eval_type, file_num = i)
#                     sh_str += s + '\n'
#     with open('./benchmark_methods_api/sh_file/' + eval_type + '_' + method + '_curl.sh', 'w') as f:
#         f.write(sh_str)

In [38]:
# for method in methods:
#     curl_sh_file(neoantigen, hla_methods_dict, 'neoantigen', method, 500)
#     curl_sh_file(hpv_vaccine, hla_methods_dict, 'hpv_vaccine', method, 500)

### anthem sh

In [39]:
# def anthem_sh(eval_data, eval_type):
#     pep_pwd = '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/test_peptide/'
#     out_pwd = '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/'

#     sh_str = ''
#     for length in range(8, 15):
#         data_length = eval_data[eval_data.Length == length]
#         if data_length.shape[0] == 0: continue
            
#         hlas = list(set(data_length.HLA))
#         for hla in tqdm(hlas):
#             data_length_hla = data_length[data_length.HLA == hla]
#             peptides = data_length_hla.peptide

#             s1 = 'python sware_b_main.py'
#             s2 = ' --length ' + str(length)
#             s3 = ' --HLA ' + hla

#             pep_filename = eval_type + '_length' + str(length) + '_' + hla
#             s4 = ' --mode prediction --fasta_file ' + pep_pwd + pep_filename + '.fasta'
#             s5 = ' --outputfile ' + pep_filename
#             s = s1 + s2 + s3 + s4 + s5
#             sh_str += s + '\n'

#             peptide_str = ''
#             for pep in peptides:
#                 peptide_str += '>' + pep + '\n' + pep + '\n'
#             with open(pep_pwd + pep_filename + '.fasta', 'w') as f:
#                 f.write(peptide_str)
#     with open('/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/' + eval_type + '.sh', 'w') as f:
#         f.write(sh_str)

In [40]:
# anthem_sh(neoantigen, 'neoantigen')
# anthem_sh(hpv_vaccine, 'hpv_vaccine')

# 其他方法结果处理

### 合并结果文件

In [41]:
def benchmark_results_pd(pwd, filenames):
    error_files = []
    for idx, filename in enumerate(filenames):
        if idx == 0:
            benchmark_pd = pd.read_csv(pwd + filename, sep = '\t')
            num_col = benchmark_pd.shape[1]
        else:
            try:
                temp = pd.read_csv(pwd + filename, sep = '\t')
            except:
                error_files.append(filename)
                continue

            if temp.shape[1] != num_col: 
                error_files.append(filename)
                continue
            else:
                benchmark_pd = pd.concat([benchmark_pd, temp], axis = 0)
    try:
        (benchmark_pd.start == 1).all()
        benchmark_pd.reset_index(drop = True, inplace = True)
        benchmark_pd.drop(columns = ['seq_num', 'start', 'end'], inplace = True)
        benchmark_pd.rename(columns = {'allele':'HLA'}, inplace = True)
    except Exception as e:
        print(e, filenames)

    return benchmark_pd, error_files

In [42]:
def merge_benchmark_with_original_data(eval_data, hla_methods_dict, eval_type = 'experimental', method = 'ann'):
    pwd = '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/output_file/' + method + '/'
    filenames = sorted(os.listdir(pwd))
    filenames = [name for name in filenames if eval_type in name]
    if eval_type == 'hpv_vaccine':
        filenames = [item for item in filenames if item.split('_')[3] in hla_methods_dict[method]]
    else:
        filenames = [item for item in filenames if item.split('_')[2] in hla_methods_dict[method]]
        
    if method == 'smm' or method == 'smmpmbec':
        filenames = [item for item in filenames if ('length12' not in item) and ('length13' not in item) and ('length14' not in item)]
    if method == 'comblib_sidney2008':
        filenames = [item for item in filenames if 'length9' in item]
    
    benchmark_pd, error_files = benchmark_results_pd(pwd, filenames)
    print(method, ':', error_files)
    merged_data = pd.merge(eval_data, benchmark_pd, on = ['peptide', 'HLA', 'length'])
    
    if 'consensus_percentile_rank' in merged_data.columns: 
        merged_data.rename(columns = {'consensus_percentile_rank':'percentile_rank'}, inplace = True)
    return merged_data, error_files

In [43]:
hpv_merged_data_dict, hpv_error_files_dict = {}, {}
for method in tqdm(methods[:-2]):
    print('----{}----'.format(method))
    hpv_merged_data_dict[method], hpv_error_files_dict[method] = merge_benchmark_with_original_data(hpv_vaccine, hla_methods_dict, 'hpv_vaccine', method)

for k, v in hpv_error_files_dict.items():
    print(k, v)

 10%|█         | 1/10 [00:00<00:01,  8.31it/s]

----ann----
ann : []
----comblib_sidney2008----
comblib_sidney2008 : []
----consensus----


 50%|█████     | 5/10 [00:00<00:00, 10.33it/s]

consensus : []
----netmhccons----
netmhccons : []
----netmhcpan_ba----
netmhcpan_ba : []
----netmhcpan_el----


 80%|████████  | 8/10 [00:00<00:00, 11.05it/s]

netmhcpan_el : []
----netmhcstabpan----
netmhcstabpan : []
----pickpocket----
pickpocket : []
----smm----


100%|██████████| 10/10 [00:00<00:00, 11.30it/s]

smm : ['hpv_vaccine_length11_HLA-B*15:01_0.output', 'hpv_vaccine_length8_HLA-B*15:01_0.output']
----smmpmbec----
smmpmbec : ['hpv_vaccine_length11_HLA-B*15:01_0.output', 'hpv_vaccine_length8_HLA-B*15:01_0.output']
ann []
comblib_sidney2008 []
consensus []
netmhccons []
netmhcpan_ba []
netmhcpan_el []
netmhcstabpan []
pickpocket []
smm ['hpv_vaccine_length11_HLA-B*15:01_0.output', 'hpv_vaccine_length8_HLA-B*15:01_0.output']
smmpmbec ['hpv_vaccine_length11_HLA-B*15:01_0.output', 'hpv_vaccine_length8_HLA-B*15:01_0.output']


In [44]:
neoantigen_merged_data_dict, neoantigen_error_files_dict = {}, {}
for method in tqdm(methods[:-2]):
    print('----{}----'.format(method))
    neoantigen_merged_data_dict[method], neoantigen_error_files_dict[method] = merge_benchmark_with_original_data(neoantigen, hla_methods_dict, 'neoantigen', method)

for k, v in neoantigen_error_files_dict.items():
    print(k, v)

  0%|          | 0/10 [00:00<?, ?it/s]

----ann----


 10%|█         | 1/10 [00:00<00:02,  3.31it/s]

ann : []
----comblib_sidney2008----
comblib_sidney2008 : []
----consensus----


 30%|███       | 3/10 [00:00<00:01,  4.03it/s]

...

netmhcpan_el : []
----netmhcstabpan----
netmhcstabpan : []
----pickpocket----


 90%|█████████ | 9/10 [00:01<00:00,  4.69it/s]

pickpocket : []
----smm----
smm : ['neoantigen_length10_HLA-B*15:17_0.output', 'neoantigen_length10_HLA-B*38:01_0.output', 'neoantigen_length10_HLA-C*03:03_0.output', 'neoantigen_length10_HLA-C*05:01_0.output', 'neoantigen_length10_HLA-C*07:01_0.output', 'neoantigen_length10_HLA-C*12:03_0.output', 'neoantigen_length11_HLA-B*15:01_0.output', 'neoantigen_length11_HLA-B*38:01_0.output', 'neoantigen_length11_HLA-B*44:02_0.output', 'neoantigen_length11_HLA-C*05:01_0.output', 'neoantigen_length8_HLA-A*31:01_0.output', 'neoantigen_length8_HLA-B*15:01_0.output', 'neoantigen_length8_HLA-B*38:01_0.output']
----smmpmbec----


100%|██████████| 10/10 [00:02<00:00,  4.88it/s]

smmpmbec : ['neoantigen_length10_HLA-B*15:17_0.output', 'neoantigen_length10_HLA-B*38:01_0.output', 'neoantigen_length10_HLA-C*03:03_0.output', 'neoantigen_length10_HLA-C*05:01_0.output', 'neoantigen_length10_HLA-C*07:01_0.output', 'neoantigen_length10_HLA-C*12:03_0.output', 'neoantigen_length11_HLA-B*15:01_0.output', 'neoantigen_length11_HLA-B*38:01_0.output', 'neoantigen_length11_HLA-B*44:02_0.output', 'neoantigen_length11_HLA-C*05:01_0.output', 'neoantigen_length8_HLA-A*31:01_0.output', 'neoantigen_length8_HLA-B*15:01_0.output', 'neoantigen_length8_HLA-B*38:01_0.output']
ann []
comblib_sidney2008 []
consensus []
netmhccons []
netmhcpan_ba []
netmhcpan_el []
netmhcstabpan []
pickpocket []
smm ['neoantigen_length10_HLA-B*15:17_0.output', 'neoantigen_length10_HLA-B*38:01_0.output', 'neoantigen_length10_HLA-C*03:03_0.output', 'neoantigen_length10_HLA-C*05:01_0.output', 'neoantigen_length10_HLA-C*07:01_0.output', 'neoantigen_length10_HLA-C*12:03_0.output', 'neoantigen_length11_HLA-B*15:0

### 结果文件处理

##### Anthem

In [45]:
def anthem_results_file(eval_type = 'neoantigen', eval_data = neoantigen):
    pwd = '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/'
    prediction_files, binder_files = [], []
    for root,dirs,files in os.walk(pwd):
        for file in files:
            if eval_type in root:
                if file[-21:] == 'prediction_result.txt':
                    prediction_files.append(os.path.join(root,file))
                else:
                    binder_files.append(os.path.join(root,file))
    print(len(prediction_files), len(binder_files), '\n', prediction_files)
    return prediction_files, binder_files

In [46]:
def find_cannot_predict_file(eval_type, prediction_files):
    with open('/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/' + eval_type + '.sh', 'r') as shf:
        sh_data = shf.readlines()
    sh_data = [sh.split(' ')[-1][:-1] for sh in sh_data]
    sh_data

    pf_data = [pf.split('/')[7] for pf in prediction_files]
    pf_data

    error_sh = set(sh_data).difference(set(pf_data))
    return [sh for sh in error_sh if sh in sh_data]

In [47]:
neoantigen_prediction_files, neoantigen_binder_files = anthem_results_file(eval_type = 'neoantigen', eval_data = neoantigen)

54 54 
 ['/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/neoantigen_length10_HLA-A*24:02/length_10_HLA-A*24:02_prediction_result.txt', '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/neoantigen_length9_HLA-C*05:01/length_9_HLA-C*05:01_prediction_result.txt', '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/neoantigen_length9_HLA-A*01:01/length_9_HLA-A*01:01_prediction_result.txt', '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/neoantigen_length9_HLA-B*18:01/length_9_HLA-B*18:01_prediction_result.txt', '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/neoantigen_length9_HLA-B*15:01/length_9_HLA-B*15:01_prediction_result.txt', '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/neoantigen_length10_HLA-A*01:01/length_10_HLA-A*01:01_prediction_result.txt', '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/neoantigen_length9_HLA-B*44:02/length_9_HLA-B*44:02_prediction_result.txt', '/home/chujunyi/5_ZY_MHC/ben

In [48]:
hpv_prediction_files, hpv_binder_files = anthem_results_file(eval_type = 'hpv_vaccine', eval_data = hpv_vaccine)

28 28 
 ['/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/hpv_vaccine_length10_HLA-A*03:01/length_10_HLA-A*03:01_prediction_result.txt', '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/hpv_vaccine_length9_HLA-B*07:02/length_9_HLA-B*07:02_prediction_result.txt', '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/hpv_vaccine_length11_HLA-B*15:01/length_11_HLA-B*15:01_prediction_result.txt', '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/hpv_vaccine_length8_HLA-A*03:01/length_8_HLA-A*03:01_prediction_result.txt', '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/hpv_vaccine_length10_HLA-A*11:01/length_10_HLA-A*11:01_prediction_result.txt', '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/hpv_vaccine_length8_HLA-B*15:01/length_8_HLA-B*15:01_prediction_result.txt', '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/output/hpv_vaccine_length8_HLA-A*11:01/length_8_HLA-A*11:01_prediction_result.txt', '/home/chujunyi/5_Z

In [49]:
find_cannot_predict_file('neoantigen', neoantigen_prediction_files), find_cannot_predict_file('hpv_vaccine', hpv_prediction_files)

(['neoantigen_length12_HLA-B*38:01',
  'neoantigen_length8_HLA-A*30:02',
  'neoantigen_length8_HLA-B*38:01',
  'neoantigen_length11_HLA-B*38:01',
  'neoantigen_length10_HLA-C*12:03',
  'neoantigen_length8_HLA-A*31:01',
  'neoantigen_length10_HLA-B*15:17',
  'neoantigen_length10_HLA-B*38:01'],
 [])

In [50]:
def anthem_results(prediction_files, binder_files, eval_data):
    res = []
    col_names = ['HLA', 'peptide', 'length', 'y_score_pred', 'score']
    all_num, all_pred_pos_num, all_pred_neg_num = 0, 0, 0
    for p_file, b_file in zip(prediction_files, binder_files):
        with open(p_file, 'r') as pf:
            pdata = pf.readlines()
        with open(b_file, 'r') as bf:
            bdata = bf.read().split('\n')
            all_num += len(bdata)

            pos_data = [item for item in bdata[1:] if item != '']
            neg_data = [item for item in bdata[1:] if item == '']
            all_pred_pos_num += len(pos_data)
            all_pred_neg_num += len(neg_data)

        for s in pdata[1:]:
            if s[:4] == 'HLA-': 
                hla = s.replace('\n', '')
            elif s[:16] == '          \t   \t\t':
                s_temp = s.split('\t')
                res.append([hla, s_temp[3], len(s_temp[3]), [0, 1][s_temp[5] == 'yes'], eval(s_temp[7])])

    assert len(res) == all_num - len(prediction_files)
    print(len(res), all_num, all_pred_pos_num, all_pred_neg_num)
    
    res_pd = pd.DataFrame(res, columns = col_names)
    print(res_pd.shape, Counter(res_pd.y_score_pred))
    
    return pd.merge(eval_data, res_pd, on = ['HLA', 'peptide', 'length'])

In [51]:
neoantigen_anthem_results = anthem_results(neoantigen_prediction_files, neoantigen_binder_files, neoantigen)
neoantigen_anthem_results

211 265 162 49
(211, 5) Counter({1: 162, 0: 49})


,peptide,HLA,Length,Mut_pos,length,label,HLA_sequence,y_pred,y_prob,y_score_pred,score
0,PSDTRQMLFY,HLA-A*01:01,10,2,10,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1.0000,1,0.999
1,LTDDRLFTCY,HLA-A*01:01,10,10,10,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1.0000,1,0.999
2,YTDFHCQYV,HLA-A*01:01,9,5,9,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1.0000,1,1.000
3,YTDFHCQYVK,HLA-A*01:01,10,5,10,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,0.9999,0,0.942
4,FSADIFFFF,HLA-A*01:01,9,6,9,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1.0000,1,0.972
...,...,...,...,...,...,...,...,...,...,...,...
206,VRINTARPV,HLA-C*06:02,9,9,9,1,YDSGYREKYRQADVNKLYLWYDSYTWAEWAYTWY,1,0.9992,0,0.911
207,FRSGLDSYV,HLA-C*06:02,9,4,9,1,YDSGYREKYRQADVNKLYLWYDSYTWAEWAYTWY,1,1.0000,1,0.957
208,AEPINIQTW,HLA-B*44:03,9,5,9,1,YYTKYREISTNTYENTAYIRYDDYTWAVLAYLSY,1,1.0000,1,0.998
209,KELEGILLL,HLA-B*44:03,9,9,9,1,YYTKYREISTNTYENTAYIRYDDYTWAVLAYLSY,1,1.0000,1,0.997


In [52]:
hpv_anthem_results = anthem_results(hpv_prediction_files, hpv_binder_files, hpv_vaccine)
hpv_anthem_results

278 306 70 208
(278, 5) Counter({0: 208, 1: 70})


,peptide,HLA,Length,length,label,HLA_sequence,y_pred,y_prob,y_score_pred,score
0,ISEYRHYCY,HLA-A*01:01,9,9,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1.0000,1,0.996
1,HGDTPTLHEY,HLA-A*01:01,10,10,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1.0000,1,0.999
2,DLQPETTDLY,HLA-A*01:01,10,10,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1.0000,1,0.995
3,AVCDKCLKFY,HLA-A*01:01,10,10,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1.0000,0,0.531
4,LKFYSKISEY,HLA-A*01:01,10,10,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,0,0.0347,0,0.953
...,...,...,...,...,...,...,...,...,...,...
273,LLMGTLGIVC,HLA-B*15:01,10,10,1,YYAMYREISTNTYESNLYLRYDSYTWAEWAYLWY,0,0.0000,0,0.197
274,CQKPLCPEE,HLA-B*15:01,9,9,1,YYAMYREISTNTYESNLYLRYDSYTWAEWAYLWY,0,0.0000,0,0.044
275,QLLRREVYDF,HLA-B*15:01,10,10,1,YYAMYREISTNTYESNLYLRYDSYTWAEWAYLWY,1,0.8305,0,0.568
276,FYSKISEY,HLA-B*15:01,8,8,1,YYAMYREISTNTYESNLYLRYDSYTWAEWAYLWY,1,0.9834,0,0.948


In [53]:
neoantigen_merged_data_dict['anthem'] = neoantigen_anthem_results
hpv_merged_data_dict['anthem'] = hpv_anthem_results

In [56]:
neoantigen_anthem_metrics = performances(neoantigen_anthem_results.label, neoantigen_anthem_results.y_score_pred, neoantigen_anthem_results.score)
hpv_anthem_metrics = performances(hpv_anthem_results.label, hpv_anthem_results.y_score_pred, hpv_anthem_results.score)
# metrics = ['roc_auc', 'sensitivity', 'specificity', 'accuracy', 'mcc', 'precision', 'recall', 'f1', 'aupr']
# summary_metrics_pd = pd.DataFrame(summary_metrics, index = metrics).T
# summary_metrics_pd
neoantigen_anthem_metrics, hpv_anthem_metrics

tn = 0, fp = 0, fn = 49, tp = 162
y_pred: 0 = 49 | 1 = 162
y_true: 0 = 0 | 1 = 211
acc=0.7678
tn = 0, fp = 0, fn = 208, tp = 70
y_pred: 0 = 208 | 1 = 70
y_true: 0 = 0 | 1 = 278
acc=0.2518


((49, 162, 0.7677725118483413), (208, 70, 0.2517985611510791))

##### 其他方法 

In [57]:
def benchmark_label_results(merged_data, eval_type = 'experimental', method = 'ann'):
    columns = merged_data.columns
    performances_ic50, performances_rank, performances_score = [None, None, None]
    if 'ic50' in columns: 
        print('-----Performances according to IC50 threshold-----')
        merged_data['y_pred_ic50'] = [[0, 1][ic50 <= 500] for ic50 in merged_data.ic50]
        performances_ic50 = performances(merged_data.label, merged_data.y_pred_ic50, merged_data.ic50)

    if 'percentile_rank' in columns: 
        print('-----Performances according to Rank threshold-----')
        merged_data['y_pred_rank'] = [[0, 1][r <= 2] for r in merged_data.percentile_rank]
        performances_rank = performances(merged_data.label, merged_data.y_pred_rank, merged_data.percentile_rank)

    if 'score' in columns: 
        print('-----Performances according to Score threshold-----')
        merged_data['y_pred_score'] = [[0, 1][score >= 0.5] for score in merged_data.score]
        performances_score = performances(merged_data.label, merged_data.y_pred_score, merged_data.score)

    return merged_data, performances_ic50, performances_rank, performances_score

In [66]:
 hpv_metrics_dict

{'1-9-4': (101, 177, 0.6366906474820144)}

In [69]:
def methods_performances_comparison(eval_type, merged_data_dict, transformer_metrics = hpv_metrics_dict['1-9-2']):
    methods_performances_ic50_dict, methods_performances_rank_dict, methods_performances_score_dict = {}, {}, {}
    transformer_according_methods_performances_dict = {}
    for method in (methods[:-1]):
        print('*** Type = {} | Method = {} ***'.format(eval_type, method))
        merged_data = merged_data_dict[method]
        transformer_according_methods_performances_dict[method] = performances(merged_data.label, merged_data.y_pred, merged_data.y_prob)
        merged_data, performances_ic50, performances_rank, performances_score = benchmark_label_results(merged_data, eval_type, method)
        methods_performances_ic50_dict[method] = performances_ic50
        if method != 'anthem':
            methods_performances_rank_dict[method] = performances_rank
        else:
            methods_performances_rank_dict[method] = performances_score
        methods_performances_score_dict[method] = performances_score

    metrics = ['FN', 'TP', 'accuracy']
    methods_performances_ic50_pd = pd.DataFrame(methods_performances_ic50_dict, index = metrics).T
    methods_performances_rank_pd = pd.DataFrame(methods_performances_rank_dict, index = metrics).T
    methods_performances_score_pd = pd.DataFrame(methods_performances_score_dict, index = metrics).T
    transformer_according_methods_performances_pd = pd.DataFrame(transformer_according_methods_performances_dict, index = metrics).T

    methods_performances_ic50_pd.loc['transformer'] = transformer_metrics
    methods_performances_rank_pd.loc['transformer'] = transformer_metrics
    methods_performances_score_pd.loc['transformer'] = transformer_metrics
    return transformer_according_methods_performances_pd, methods_performances_ic50_pd, methods_performances_rank_pd, methods_performances_score_pd

In [70]:
neoantigen_transformer_according_methods_performances_pd, neoantigen_methods_performances_ic50_pd, neoantigen_methods_performances_rank_pd, neoantigen_methods_performances_score_pd = methods_performances_comparison(
eval_type = 'neoantigen_vaccine', 
merged_data_dict = neoantigen_merged_data_dict, 
transformer_metrics = neoantigen_metrics_dict['1-9-2']) 

*** Type = neoantigen_vaccine | Method = ann ***
tn = 0, fp = 0, fn = 12, tp = 206
y_pred: 0 = 12 | 1 = 206
y_true: 0 = 0 | 1 = 218
acc=0.9450
-----Performances according to IC50 threshold-----
tn = 0, fp = 0, fn = 41, tp = 177
y_pred: 0 = 41 | 1 = 177
y_true: 0 = 0 | 1 = 218
acc=0.8119
-----Performances according to Rank threshold-----
tn = 0, fp = 0, fn = 26, tp = 192
y_pred: 0 = 26 | 1 = 192
y_true: 0 = 0 | 1 = 218
acc=0.8807
*** Type = neoantigen_vaccine | Method = comblib_sidney2008 ***
tn = 0, fp = 0, fn = 4, tp = 79
y_pred: 0 = 4 | 1 = 79
y_true: 0 = 0 | 1 = 83
acc=0.9518
-----Performances according to IC50 threshold-----
tn = 0, fp = 0, fn = 0, tp = 83
y_pred: 0 = 0 | 1 = 83
y_true: 0 = 0 | 1 = 83
acc=1.0000
-----Performances according to Rank threshold-----
tn = 0, fp = 0, fn = 47, tp = 36
y_pred: 0 = 47 | 1 = 36
y_true: 0 = 0 | 1 = 83
acc=0.4337
*** Type = neoantigen_vaccine | Method = consensus ***
tn = 0, fp = 0, fn = 12, tp = 205
y_pred: 0 = 12 | 1 = 205
y_true: 0 = 0 | 1 

In [71]:
neoantigen_methods_performances_ic50_pd

,FN,TP,accuracy
ann,41.0,177.0,0.811927
comblib_sidney2008,0.0,83.0,1.000000
consensus,None,None,None
netmhccons,26.0,194.0,0.881818
netmhcpan_ba,38.0,183.0,0.828054
netmhcpan_el,None,None,None
netmhcstabpan,None,None,None
pickpocket,83.0,138.0,0.624434
smm,27.0,171.0,0.863636
smmpmbec,26.0,173.0,0.869347


In [72]:
neoantigen_methods_performances_score_pd

,FN,TP,accuracy
ann,None,None,None
comblib_sidney2008,None,None,None
consensus,None,None,None
netmhccons,None,None,None
netmhcpan_ba,None,None,None
netmhcpan_el,82.0,139.0,0.628959
netmhcstabpan,33.0,188.0,0.850679
pickpocket,None,None,None
smm,None,None,None
smmpmbec,None,None,None


In [73]:
neoantigen_methods_performances_rank_pd

,FN,TP,accuracy
ann,26.0,192.0,0.880734
comblib_sidney2008,47.0,36.0,0.433735
consensus,40.0,177.0,0.815668
netmhccons,27.0,193.0,0.877273
netmhcpan_ba,17.0,204.0,0.923077
netmhcpan_el,22.0,199.0,0.900452
netmhcstabpan,58.0,163.0,0.737557
pickpocket,43.0,178.0,0.805430
smm,34.0,164.0,0.828283
smmpmbec,39.0,160.0,0.804020


In [56]:
neoantigen_transformer_according_methods_performances_pd

,FN,TP,accuracy
ann,7.0,211.0,0.967890
comblib_sidney2008,1.0,82.0,0.987952
consensus,7.0,210.0,0.967742
netmhccons,7.0,213.0,0.968182
netmhcpan_ba,7.0,214.0,0.968326
netmhcpan_el,7.0,214.0,0.968326
netmhcstabpan,7.0,214.0,0.968326
pickpocket,7.0,214.0,0.968326
smm,5.0,193.0,0.974747
smmpmbec,5.0,194.0,0.974874


In [74]:
hpv_transformer_according_methods_performances_pd, hpv_methods_performances_ic50_pd, hpv_methods_performances_rank_pd, hpv_methods_performances_score_pd = methods_performances_comparison(
eval_type = 'hpv_vaccine', 
merged_data_dict = hpv_merged_data_dict, 
transformer_metrics = hpv_metrics_dict['1-9-2']) 

*** Type = hpv_vaccine | Method = ann ***
tn = 0, fp = 0, fn = 101, tp = 177
y_pred: 0 = 101 | 1 = 177
y_true: 0 = 0 | 1 = 278
acc=0.6367
-----Performances according to IC50 threshold-----
tn = 0, fp = 0, fn = 225, tp = 53
y_pred: 0 = 225 | 1 = 53
y_true: 0 = 0 | 1 = 278
acc=0.1906
-----Performances according to Rank threshold-----
tn = 0, fp = 0, fn = 211, tp = 67
y_pred: 0 = 211 | 1 = 67
y_true: 0 = 0 | 1 = 278
acc=0.2410
*** Type = hpv_vaccine | Method = comblib_sidney2008 ***
tn = 0, fp = 0, fn = 10, tp = 30
y_pred: 0 = 10 | 1 = 30
y_true: 0 = 0 | 1 = 40
acc=0.7500
-----Performances according to IC50 threshold-----
tn = 0, fp = 0, fn = 0, tp = 40
y_pred: 0 = 0 | 1 = 40
y_true: 0 = 0 | 1 = 40
acc=1.0000
-----Performances according to Rank threshold-----
tn = 0, fp = 0, fn = 34, tp = 6
y_pred: 0 = 34 | 1 = 6
y_true: 0 = 0 | 1 = 40
acc=0.1500
*** Type = hpv_vaccine | Method = consensus ***
tn = 0, fp = 0, fn = 100, tp = 171
y_pred: 0 = 100 | 1 = 171
y_true: 0 = 0 | 1 = 271
acc=0.6310


In [75]:
Counter(hpv_vaccine['y_pred'])

Counter({1: 177, 0: 101})

In [76]:
hpv_transformer_according_methods_performances_pd

,FN,TP,accuracy
ann,101.0,177.0,0.636691
comblib_sidney2008,10.0,30.0,0.750000
consensus,100.0,171.0,0.630996
netmhccons,101.0,177.0,0.636691
netmhcpan_ba,101.0,177.0,0.636691
netmhcpan_el,101.0,177.0,0.636691
netmhcstabpan,101.0,177.0,0.636691
pickpocket,101.0,177.0,0.636691
smm,92.0,158.0,0.632000
smmpmbec,92.0,158.0,0.632000


In [77]:
hpv_methods_performances_ic50_pd

,FN,TP,accuracy
ann,225.0,53.0,0.190647
comblib_sidney2008,0.0,40.0,1.000000
consensus,None,None,None
netmhccons,206.0,72.0,0.258993
netmhcpan_ba,225.0,53.0,0.190647
netmhcpan_el,None,None,None
netmhcstabpan,None,None,None
pickpocket,240.0,38.0,0.136691
smm,173.0,77.0,0.308000
smmpmbec,175.0,75.0,0.300000


In [78]:
hpv_methods_performances_score_pd

,FN,TP,accuracy
ann,None,None,None
comblib_sidney2008,None,None,None
consensus,None,None,None
netmhccons,None,None,None
netmhcpan_ba,None,None,None
netmhcpan_el,256.0,22.0,0.079137
netmhcstabpan,116.0,162.0,0.582734
pickpocket,None,None,None
smm,None,None,None
smmpmbec,None,None,None


In [79]:
hpv_methods_performances_rank_pd

,FN,TP,accuracy
ann,211.0,67.0,0.241007
comblib_sidney2008,34.0,6.0,0.150000
consensus,226.0,45.0,0.166052
netmhccons,191.0,87.0,0.312950
netmhcpan_ba,177.0,101.0,0.363309
netmhcpan_el,163.0,115.0,0.413669
netmhcstabpan,182.0,96.0,0.345324
pickpocket,213.0,65.0,0.233813
smm,185.0,65.0,0.260000
smmpmbec,187.0,63.0,0.252000


In [80]:
neoantigen[neoantigen.y_prob < 0.5]

,peptide,HLA,Length,Mut_pos,length,label,HLA_sequence,y_pred,y_prob
31,KLILWRGLK,HLA-A*02:01,9,2,9,1,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY,0,0.0000
33,KVCCCQILL,HLA-A*02:01,9,3,9,1,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY,0,0.3291
51,LLYPHSLSS,HLA-A*02:01,9,6,9,1,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY,0,0.0045
70,GIAARVKNWL,HLA-A*02:01,10,7,10,1,YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY,0,0.0013
127,VQFLSQVLWSR,HLA-A*11:01,11,6,11,1,YYAMYQENVAQTDVDTLYIIYRDYTWAAQAYRWY,0,0.0110
160,KGTQRILAAL,HLA-B*07:02,10,7,10,1,YYSEYRNIYAQTDESNLYLSYDYYTWAERAYEWY,0,0.4114
182,LLCSSCWDF,HLA-B*15:01,9,4,9,1,YYAMYREISTNTYESNLYLRYDSYTWAEWAYLWY,0,0.0456
186,KAFIYKSDFV,HLA-B*15:17,10,9,10,1,YYAMYRENMASTYENIAYLRYHDYTWAELAYLWY,0,0.0063
199,HNAYHSIEWAI,HLA-B*38:01,11,6,11,1,YYSEYRNICTNTYENIAYLRYNFYTWAVLTYTWY,0,0.0000
215,ATLVAAFTL,HLA-C*07:01,9,2,9,1,YDSGYRENYRQADVSNLYLRYDSYTLAALAYTWY,0,0.0046
